In [1]:
%run model_functions.ipynb

In [2]:
words = pd.read_csv('data/raw/words.csv',
                          encoding='ISO--8859-1', na_filter=False)
display(len(words))
words.head()

204834

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id
0,0,Latest,late,AJS,,,0,a1e-fragment01
1,1,corporate,corporate,AJ0,,,0,a1e-fragment01
2,2,unbundler,unbundler,NN1,,,0,a1e-fragment01
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01
4,4,laid-back,laid-back,AJ0,,,0,a1e-fragment01


In [26]:
words[words['seg_type']=='met']

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,sentence_num,lowercase,offset
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01,0,reveals,4
5,5,approach,approach,NN1,mrw,met,0,a1e-fragment01,0,approach,6
10,10,leading,lead,VVG,mrw,met,0,a1e-fragment01,0,leading,11
19,19,to,to,PRP,mrw,met,0,a1e-fragment01,0,to,20
39,39,made,make,VVN,mrw,met,2,a1e-fragment01,2,made,15
41,41,fatal,fatal,AJ0,mrw,met,2,a1e-fragment01,2,fatal,17
43,43,in,in,PRP,mrw,met,2,a1e-fragment01,2,in,19
63,63,target,target,NN1,mrw,met,3,a1e-fragment01,3,target,7
68,68,head,head,NN1,mrw,met,4,a1e-fragment01,4,head,3
72,72,through,through,PRP,mrw,met,4,a1e-fragment01,4,through,7


Save sentence on each newline.

```python
with open('data/raw/sentences.txt', 'w') as f:
    for sentence_id, group in words.groupby('sentence_id'):
        if not sentence_id % 100:
            print('Finding dependencies for sentence {}'.format(sentence_id))
            
        sentence = ' '.join(list(group['word']))
        
        f.write(sentence + '\n')
            
    print('Done!')
```

Assign sentence with number so we can match them later to the frame parsing.

In [3]:
sentence_nums = []
sentence_num = 0
for sentence_id, group in words.groupby('sentence_id'):
    sentence_nums.append([sentence_num, sentence_id])
    sentence_num += 1    

In [4]:
words['sentence_num'] = words.merge(pd.DataFrame(sentence_nums, columns=['sentence_num', 'sentence_id']))['sentence_num']
words.sample(10)

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,sentence_num
5796,5796,16,16,CRD,,,269,a1j-fragment34,269
195678,195678,house,house,NN1,,,14503,kbw-fragment42,14476
71632,71632,a,a,AT0,,,3651,b17-fragment02,3645
110281,110281,an,an,AT0,,,5841,ccw-fragment04,5835
6263,6263,&,&,CJC,,,295,a1k-fragment02,295
204748,204748,here,here,AV0,,,16172,kcv-fragment42,16133
59111,59111,health,health,NN1,,,3124,al5-fragment03,3118
54847,54847,had,have,VHD,mrw,met,2910,ahf-fragment63,2904
116482,116482,particular,particular,AJ0,,,6197,clp-fragment01,6190
72998,72998,also,also,AV0,,,3715,b17-fragment02,3709


In [5]:
words['lowercase'] = words['word'].str.lower()
words.head()

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,sentence_num,lowercase
0,0,Latest,late,AJS,,,0,a1e-fragment01,0,latest
1,1,corporate,corporate,AJ0,,,0,a1e-fragment01,0,corporate
2,2,unbundler,unbundler,NN1,,,0,a1e-fragment01,0,unbundler
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01,0,reveals
4,4,laid-back,laid-back,AJ0,,,0,a1e-fragment01,0,laid-back


Run the following!

```python
python -m sesame.targetid --mode predict \
                           --model_name fn1.7-pretrained-targetid \
                           --raw_input ../data/raw/sentences.txt
```

```python
python -m sesame.frameid --mode predict \
                          --model_name fn1.7-pretrained-frameid \
                          --raw_input logs/fn1.7-pretrained-targetid/predicted-targets.conll
```

In [6]:
tmp = (
    pd.read_csv('open-sesame/logs/fn1.7-pretrained-frameid/predicted-frames.conll',
                sep='\t',
                header=None)
    .drop_duplicates()
    .rename(columns={
        0:'offset',
        1:'token',
        6:'sentence_num',
        13:'frame'
    })
)
display(len(tmp))
tmp.head(50)

233392

,offset,token,2,3,4,5,sentence_num,7,8,9,10,11,12,frame,14
0,1,latest,_,UNK,_,NNP,0,_,_,_,_,_,_,_,O
1,2,corporate,_,corporate,_,JJ,0,_,_,_,_,_,_,_,O
2,3,unk,_,UNK,_,NN,0,_,_,_,_,_,_,_,O
3,4,reveals,_,reveal,_,VBZ,0,_,_,_,_,_,reveal.v,Reveal_secret,O
4,5,laid-back,_,UNK,_,JJ,0,_,_,_,_,_,_,_,O
5,6,approach,_,approach,_,NN,0,_,_,_,_,_,_,_,O
6,7,roland,_,UNK,_,NNP,0,_,_,_,_,_,_,_,O
7,8,franklin,_,UNK,_,NNP,0,_,_,_,_,_,_,_,O
8,9,who,_,who,_,WP,0,_,_,_,_,_,_,_,O
9,10,is,_,be,_,VBZ,0,_,_,_,_,_,_,_,O


In [7]:
tmp['has_duplicate'] = tmp.duplicated(subset=['offset', 'sentence_num'], keep=False)
tmp.head(50)

,offset,token,2,3,4,5,sentence_num,7,8,9,10,11,12,frame,14,has_duplicate
0,1,latest,_,UNK,_,NNP,0,_,_,_,_,_,_,_,O,False
1,2,corporate,_,corporate,_,JJ,0,_,_,_,_,_,_,_,O,False
2,3,unk,_,UNK,_,NN,0,_,_,_,_,_,_,_,O,False
3,4,reveals,_,reveal,_,VBZ,0,_,_,_,_,_,reveal.v,Reveal_secret,O,True
4,5,laid-back,_,UNK,_,JJ,0,_,_,_,_,_,_,_,O,False
5,6,approach,_,approach,_,NN,0,_,_,_,_,_,_,_,O,False
6,7,roland,_,UNK,_,NNP,0,_,_,_,_,_,_,_,O,False
7,8,franklin,_,UNK,_,NNP,0,_,_,_,_,_,_,_,O,False
8,9,who,_,who,_,WP,0,_,_,_,_,_,_,_,O,False
9,10,is,_,be,_,VBZ,0,_,_,_,_,_,_,_,O,False


In [8]:
frames = (
    tmp[(tmp['frame']!='_') | (tmp['has_duplicate']==False)]
    [['offset', 'token', 'sentence_num', 'frame']]
    .sort_values(by=['sentence_num', 'offset'])
)
display(len(frames))
frames.head(50)

194239

,offset,token,sentence_num,frame
0,1,latest,0,_
1,2,corporate,0,_
2,3,unk,0,_
3,4,reveals,0,Reveal_secret
4,5,laid-back,0,_
5,6,approach,0,_
6,7,roland,0,_
7,8,franklin,0,_
8,9,who,0,_
9,10,is,0,_


In [9]:
len(frames[frames['frame']!='_'])/len(frames)

0.21855034261914447

Assign new sentence ID everytime word ID starts back at 1. Should match original sentence ID assignation as Frames is parsed on all words.

In [10]:
words['offset'] = words.groupby('sentence_id').cumcount() + 1
display(len(words))
words.head(50)

204834

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,sentence_num,lowercase,offset
0,0,Latest,late,AJS,,,0,a1e-fragment01,0,latest,1
1,1,corporate,corporate,AJ0,,,0,a1e-fragment01,0,corporate,2
2,2,unbundler,unbundler,NN1,,,0,a1e-fragment01,0,unbundler,3
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01,0,reveals,4
4,4,laid-back,laid-back,AJ0,,,0,a1e-fragment01,0,laid-back,5
5,5,approach,approach,NN1,mrw,met,0,a1e-fragment01,0,approach,6
6,6,Roland,roland,NP0,,,0,a1e-fragment01,0,roland,7
7,7,Franklin,franklin,NP0,,,0,a1e-fragment01,0,franklin,8
8,8,who,who,PNQ,,,0,a1e-fragment01,0,who,9
9,9,is,be,VBZ,,,0,a1e-fragment01,0,is,10


In [11]:
tmp = words.merge(pd.read_csv('data/raw/sentences.csv',
            encoding='ISO--8859-1', na_filter=False),
            left_on='sentence_id',
            right_on='sentence_id'
           )
display(len(tmp))
tmp.head()

204834

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,sentence_num,lowercase,offset,paragraph_id,sentence
0,0,Latest,late,AJS,,,0,a1e-fragment01,0,latest,1,-1,"Latest corporate unbundler reveals laid-back approach: Roland Franklin, who is leading a 697m pound break-up bid for DRG, talks to Frank Kane"
1,1,corporate,corporate,AJ0,,,0,a1e-fragment01,0,corporate,2,-1,"Latest corporate unbundler reveals laid-back approach: Roland Franklin, who is leading a 697m pound break-up bid for DRG, talks to Frank Kane"
2,2,unbundler,unbundler,NN1,,,0,a1e-fragment01,0,unbundler,3,-1,"Latest corporate unbundler reveals laid-back approach: Roland Franklin, who is leading a 697m pound break-up bid for DRG, talks to Frank Kane"
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01,0,reveals,4,-1,"Latest corporate unbundler reveals laid-back approach: Roland Franklin, who is leading a 697m pound break-up bid for DRG, talks to Frank Kane"
4,4,laid-back,laid-back,AJ0,,,0,a1e-fragment01,0,laid-back,5,-1,"Latest corporate unbundler reveals laid-back approach: Roland Franklin, who is leading a 697m pound break-up bid for DRG, talks to Frank Kane"


In [12]:
words_with_frames = tmp.merge(frames, how='left')[['word_id', 'word', 'paragraph_id', 'text_id', 'frame']]
words_with_frames['frame'] = words_with_frames['frame'].fillna('_')
words_with_frames.head(50)

,word_id,word,paragraph_id,text_id,frame
0,0,Latest,-1,a1e-fragment01,_
1,1,corporate,-1,a1e-fragment01,_
2,2,unbundler,-1,a1e-fragment01,_
3,3,reveals,-1,a1e-fragment01,Reveal_secret
4,4,laid-back,-1,a1e-fragment01,_
5,5,approach,-1,a1e-fragment01,_
6,6,Roland,-1,a1e-fragment01,_
7,7,Franklin,-1,a1e-fragment01,_
8,8,who,-1,a1e-fragment01,_
9,9,is,-1,a1e-fragment01,_


In [13]:
from collections import Counter

gwc_list = []
for text_id, group in words_with_frames.groupby('paragraph_id'):
    counts = dict(Counter(group['frame']))
    for row in group.itertuples():
        if row.paragraph_id == -1 or row.frame == '_':
            gwc_list.append((row.word_id, -1))
        else:
            gwc_list.append((
                row.word_id,
                (counts[row.frame] - 1)/len(group[group['frame']!='_'])
            ))  # Don't count itself

In [14]:
gwc = words_with_frames.merge(pd.DataFrame(gwc_list, columns=['word_id', 'gwc']),
                              left_on='word_id', right_on='word_id'
)
display(len(gwc))
display(gwc.sample(50))
len(gwc[(gwc['gwc']!=-1) & (gwc['gwc']!=0)])

204834

,word_id,word,paragraph_id,text_id,frame,gwc
17802,17802,as,298,a3c-fragment05,_,-1.000000
127546,127546,vengeance,2024,ea7-fragment03,Revenge,0.000000
102094,102094,Dalgliesh,1613,c8t-fragment01,_,-1.000000
72701,72701,and,1140,b17-fragment02,_,-1.000000
154779,154779,They,2557,g0l-fragment01,_,-1.000000
166029,166029,down,-1,kb7-fragment48,_,-1.000000
60012,60012,question,1048,alp-fragment01,Questioning,0.000000
32974,32974,light-hearted,514,a8m-fragment02,_,-1.000000
19024,19024,fall,318,a3e-fragment03,_,-1.000000
94911,94911,elegant,1362,bmw-fragment09,_,-1.000000


8298

In [15]:
gwc.to_csv('data/words_with_frames.csv', index=False)

## Evaluation

In [16]:
%run model_functions.ipynb

In [185]:
genre = 'academic'
name = 'GWC'
do_experiment(
    genre=genre,
    name=name,
    want_gwc=True
)

academic
-----------(25360, 7328)


,word_id,word_x,lemma,word_type,function,seg_type,sentence_id,text_id_x,text_tag,genre,word_y,paragraph_id,text_id_y,frame,gwc
0,25071,DRESS,dress,NN1,,,1267,a6u-fragment02,a6u,academic,DRESS,-1,a6u-fragment02,_,-1.000000
1,25072,HANGS,hang,VVZ,,,1267,a6u-fragment02,a6u,academic,HANGS,-1,a6u-fragment02,_,-1.000000
2,25073,HERE',here',NP0,,,1267,a6u-fragment02,a6u,academic,HERE',-1,a6u-fragment02,_,-1.000000
3,25074,DE-FROCKING,de-frock,VVG,mrw,met,1267,a6u-fragment02,a6u,academic,DE-FROCKING,-1,a6u-fragment02,_,-1.000000
4,25076,KAHLO,kahlo,NP0,,,1267,a6u-fragment02,a6u,academic,KAHLO,-1,a6u-fragment02,_,-1.000000
5,25077,CULT,cult,NN1,,,1267,a6u-fragment02,a6u,academic,CULT,-1,a6u-fragment02,_,-1.000000
6,25078,Oriana,oriana,NP0-NN1,,,1268,a6u-fragment02,a6u,academic,Oriana,-1,a6u-fragment02,_,-1.000000
7,25079,Baddeley,baddeley,NP0,,,1268,a6u-fragment02,a6u,academic,Baddeley,-1,a6u-fragment02,_,-1.000000
8,25083,witnessed,witness,VVN,mrw,met,1269,a6u-fragment02,a6u,academic,witnessed,412,a6u-fragment02,_,-1.000000
9,25085,shift,shift,NN1-VVB,,,1269,a6u-fragment02,a6u,academic,shift,412,a6u-fragment02,Change_position_on_a_scale,0.013072


25360 25360
(25360, 7329)
Fitting 10 folds for each of 40 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:   

[LibLinear]academic
----(8626, 7328)


,word_id,word_x,lemma,word_type,function,seg_type,sentence_id,text_id_x,text_tag,genre,word_y,paragraph_id,text_id_y,frame,gwc
0,61516,BRINGING,bring,VVG,mrw,met,3232,amm-fragment02,amm,academic,BRINGING,-1,amm-fragment02,_,-1.000000
1,61517,FOSSILS,fossil,NN2,,,3232,amm-fragment02,amm,academic,FOSSILS,-1,amm-fragment02,_,-1.000000
2,61518,BACK,back,AVP,mrw,met,3232,amm-fragment02,amm,academic,BACK,-1,amm-fragment02,_,-1.000000
3,61520,LIFE,life,NN1,mrw,met,3232,amm-fragment02,amm,academic,LIFE,-1,amm-fragment02,_,-1.000000
4,61523,chapter,chapter,NN1,,,3233,amm-fragment02,amm,academic,chapter,1066,amm-fragment02,Part_ordered_segments,0.000000
5,61527,ways,way,NN2,mrw,met,3233,amm-fragment02,amm,academic,ways,1066,amm-fragment02,Means,0.018759
6,61530,palaeontologists,palaeontologist,NN2,,,3233,amm-fragment02,amm,academic,palaeontologists,1066,amm-fragment02,_,-1.000000
7,61531,determine,determine,VVB,,,3233,amm-fragment02,amm,academic,determine,1066,amm-fragment02,_,-1.000000
8,61533,way,way,NN1,mrw,met,3233,amm-fragment02,amm,academic,way,1066,amm-fragment02,Means,0.018759
9,61534,fossil,fossil,NN1,,,3233,amm-fragment02,amm,academic,fossil,1066,amm-fragment02,_,-1.000000


8626 8626
(8626, 7329)
{'False': {'precision': 0.81195079086116, 'recall': 0.6165480427046264, 'f1-score': 0.7008849557522124, 'support': 6744}, 'True': {'precision': 0.2621968616262482, 'recall': 0.48831030818278426, 'f1-score': 0.34119175793577133, 'support': 1882}, 'micro avg': {'precision': 0.5885694412242058, 'recall': 0.5885694412242058, 'f1-score': 0.5885694412242058, 'support': 8626}, 'macro avg': {'precision': 0.537073826243704, 'recall': 0.5524291754437053, 'f1-score': 0.5210383568439919, 'support': 8626}, 'weighted avg': {'precision': 0.6920067965625158, 'recall': 0.5885694412242058, 'f1-score': 0.6224079561822446, 'support': 8626}}


In [18]:
genre = 'conversation'
name = 'GWC'
do_experiment(
    genre=genre,
    name=name,
    want_gwc=True
)

conversation
------(15145, 2874)


,word_id,word_x,lemma,word_type,function,seg_type,sentence_id,text_id_x,text_tag,genre,word_y,paragraph_id,text_id_y,frame,gwc
0,156258,actually,actually,AV0,,,8721,kb7-fragment10,kb7,conversation,actually,-1,kb7-fragment10,_,-1.0
1,156260,new,new,AJ0,,,8721,kb7-fragment10,kb7,conversation,new,-1,kb7-fragment10,Age,-1.0
2,156261,house,house,NN1,,,8721,kb7-fragment10,kb7,conversation,house,-1,kb7-fragment10,Buildings,-1.0
3,156267,market,market,NN1,mrw,met,8722,kb7-fragment10,kb7,conversation,market,-1,kb7-fragment10,_,-1.0
4,156269,so,so,AV0,,,8722,kb7-fragment10,kb7,conversation,so,-1,kb7-fragment10,_,-1.0
5,156270,low,low,AJ0,mrw,met,8722,kb7-fragment10,kb7,conversation,low,-1,kb7-fragment10,Position_on_a_scale,-1.0
6,156273,decided,decide,VVD,,,8722,kb7-fragment10,kb7,conversation,decided,-1,kb7-fragment10,Deciding,-1.0
7,156276,well,well,AV0,,,8722,kb7-fragment10,kb7,conversation,well,-1,kb7-fragment10,_,-1.0
8,156280,think,think,VVI,,,8722,kb7-fragment10,kb7,conversation,think,-1,kb7-fragment10,Opinion,-1.0
9,156283,get,get,VVI,mrw,met,8722,kb7-fragment10,kb7,conversation,get,-1,kb7-fragment10,_,-1.0


15145 15145
(15145, 2875)
Fitting 10 folds for each of 40 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0898s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.2403s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Batch computation too slow (2.9413s.) Setting batch_size=1.
[Parallel(n_jobs=-1)]: Done 182 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 198 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 211 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 239 tasks      | elapsed:  

[LibLinear]conversation
--(2650, 2874)


,word_id,word_x,lemma,word_type,function,seg_type,sentence_id,text_id_x,text_tag,genre,word_y,paragraph_id,text_id_y,frame,gwc
0,178401,speak,speak,VVI,,,11759,kbh-fragment02,kbh,conversation,speak,-1,kbh-fragment02,Statement,-1.0
1,178407,speak,speak,VVI,,,11761,kbh-fragment02,kbh,conversation,speak,-1,kbh-fragment02,Statement,-1.0
2,178411,very,very,AV0,,,11761,kbh-fragment02,kbh,conversation,very,-1,kbh-fragment02,_,-1.0
3,178412,relaxed,relaxed,AJ0,,,11761,kbh-fragment02,kbh,conversation,relaxed,-1,kbh-fragment02,_,-1.0
4,178416,like,like,VVI,,,11762,kbh-fragment02,kbh,conversation,like,-1,kbh-fragment02,Similarity,-1.0
5,178418,go,go,VVI,mrw,met,11762,kbh-fragment02,kbh,conversation,go,-1,kbh-fragment02,Motion,-1.0
6,178421,week,week,NN1,,,11762,kbh-fragment02,kbh,conversation,week,-1,kbh-fragment02,Calendric_unit,-1.0
7,178423,silence,silence,NN1,,,11762,kbh-fragment02,kbh,conversation,silence,-1,kbh-fragment02,_,-1.0
8,178425,know,know,VVB,,,11763,kbh-fragment02,kbh,conversation,know,-1,kbh-fragment02,Awareness,-1.0
9,178429,stay,stay,VVI,mrw,met,11763,kbh-fragment02,kbh,conversation,stay,-1,kbh-fragment02,Residence,-1.0


2650 2650
(2650, 2875)
{'False': {'precision': 0.9050337311883757, 'recall': 0.7364864864864865, 'f1-score': 0.8121071012805587, 'support': 2368}, 'True': {'precision': 0.13692946058091288, 'recall': 0.35106382978723405, 'f1-score': 0.19701492537313434, 'support': 282}, 'micro avg': {'precision': 0.6954716981132075, 'recall': 0.6954716981132075, 'f1-score': 0.6954716981132075, 'support': 2650}, 'macro avg': {'precision': 0.5209815958846443, 'recall': 0.5437751581368603, 'f1-score': 0.5045610133268466, 'support': 2650}, 'weighted avg': {'precision': 0.8232958427690155, 'recall': 0.6954716981132075, 'f1-score': 0.7466520093538064, 'support': 2650}}


In [19]:
genre = 'fiction'
name = 'GWC'
do_experiment(
    genre=genre,
    name=name,
    want_gwc=True
)

fiction
-----------(17879, 6033)


,word_id,word_x,lemma,word_type,function,seg_type,sentence_id,text_id_x,text_tag,genre,word_y,paragraph_id,text_id_y,frame,gwc
0,38264,boy,boy,NN1,,,1947,ab9-fragment03,ab9,fiction,boy,628,ab9-fragment03,People_by_age,0.054054
1,38267,dog,dog,NN1,,,1947,ab9-fragment03,ab9,fiction,dog,628,ab9-fragment03,Animals,0.027027
2,38268,skirted,skirt,VVD,,,1947,ab9-fragment03,ab9,fiction,skirted,628,ab9-fragment03,_,-1.000000
3,38270,opposite,opposite,AJ0,,,1947,ab9-fragment03,ab9,fiction,opposite,628,ab9-fragment03,Non-gradable_proximity,0.000000
4,38271,sides,side,NN2,,,1947,ab9-fragment03,ab9,fiction,sides,628,ab9-fragment03,Taking_sides,0.000000
5,38274,puddle,puddle,NN1,,,1947,ab9-fragment03,ab9,fiction,puddle,628,ab9-fragment03,_,-1.000000
6,38276,boy,boy,NN1,,,1947,ab9-fragment03,ab9,fiction,boy,628,ab9-fragment03,People_by_age,0.054054
7,38279,tall,tall,AJ0,,,1947,ab9-fragment03,ab9,fiction,tall,628,ab9-fragment03,_,-1.000000
8,38280,fourteen-year-old,fourteen-year-old,AJ0-NN1,,,1947,ab9-fragment03,ab9,fiction,fourteen-year-old,628,ab9-fragment03,_,-1.000000
9,38281,labouring,labour,VVG,mrw,met,1947,ab9-fragment03,ab9,fiction,labouring,628,ab9-fragment03,_,-1.000000


17879 17879
(17879, 6034)
Fitting 11 folds for each of 40 candidates, totalling 440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1503s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.1135s.) Setting batch_size=1.
[Parallel(n_jobs=-1)]: Done 171 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 186 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed:   36.4s
[Parallel(n_jo

[LibLinear]fiction
--(3361, 6033)


,word_id,word_x,lemma,word_type,function,seg_type,sentence_id,text_id_x,text_tag,genre,word_y,paragraph_id,text_id_y,frame,gwc
0,97775,Grant,grant,NP0,,,4888,bpa-fragment14,bpa,fiction,Grant,1444,bpa-fragment14,_,-1.0
1,97776,remains,remains,NN2,,,4888,bpa-fragment14,bpa,fiction,remains,1444,bpa-fragment14,_,-1.0
2,97778,shovelled,shovel,VVN,,,4888,bpa-fragment14,bpa,fiction,shovelled,1444,bpa-fragment14,_,-1.0
3,97781,canvas,canvas,NN1,,,4888,bpa-fragment14,bpa,fiction,canvas,1444,bpa-fragment14,_,-1.0
4,97782,bag,bag,NN1,,,4888,bpa-fragment14,bpa,fiction,bag,1444,bpa-fragment14,_,-1.0
5,97783,Tense,tense,VVB,mrw,met,4889,bpa-fragment14,bpa,fiction,Tense,1444,bpa-fragment14,_,-1.0
6,97784,facing,face,VVG,,,4889,bpa-fragment14,bpa,fiction,facing,1444,bpa-fragment14,_,-1.0
7,97785,outwards,outwards,AV0,,,4889,bpa-fragment14,bpa,fiction,outwards,1444,bpa-fragment14,_,-1.0
8,97786,Forster,forster,NP0,,,4889,bpa-fragment14,bpa,fiction,Forster,1444,bpa-fragment14,_,-1.0
9,97787,Lawton,lawton,NP0,,,4889,bpa-fragment14,bpa,fiction,Lawton,1444,bpa-fragment14,_,-1.0


3361 3361
(3361, 6034)
{'False': {'precision': 0.8548450372695174, 'recall': 0.7680648572435671, 'f1-score': 0.8091347939101374, 'support': 2837}, 'True': {'precision': 0.1896551724137931, 'recall': 0.29389312977099236, 'f1-score': 0.2305389221556886, 'support': 524}, 'micro avg': {'precision': 0.6941386492115442, 'recall': 0.6941386492115442, 'f1-score': 0.6941386492115442, 'support': 3361}, 'macro avg': {'precision': 0.5222501048416552, 'recall': 0.5309789935072797, 'f1-score': 0.519836858032913, 'support': 3361}, 'weighted avg': {'precision': 0.7511379592616627, 'recall': 0.6941386492115442, 'f1-score': 0.7189282372902829, 'support': 3361}}


In [20]:
genre = 'newspapers'
name = 'GWC'
do_experiment(
    genre=genre,
    name=name,
    want_gwc=True
)

newspapers
------(18527, 7691)


,word_id,word_x,lemma,word_type,function,seg_type,sentence_id,text_id_x,text_tag,genre,word_y,paragraph_id,text_id_y,frame,gwc
0,0,Latest,late,AJS,,,0,a1e-fragment01,a1e,newspapers,Latest,-1,a1e-fragment01,_,-1.0
1,1,corporate,corporate,AJ0,,,0,a1e-fragment01,a1e,newspapers,corporate,-1,a1e-fragment01,_,-1.0
2,2,unbundler,unbundler,NN1,,,0,a1e-fragment01,a1e,newspapers,unbundler,-1,a1e-fragment01,_,-1.0
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01,a1e,newspapers,reveals,-1,a1e-fragment01,Reveal_secret,-1.0
4,4,laid-back,laid-back,AJ0,,,0,a1e-fragment01,a1e,newspapers,laid-back,-1,a1e-fragment01,_,-1.0
5,5,approach,approach,NN1,mrw,met,0,a1e-fragment01,a1e,newspapers,approach,-1,a1e-fragment01,_,-1.0
6,6,Roland,roland,NP0,,,0,a1e-fragment01,a1e,newspapers,Roland,-1,a1e-fragment01,_,-1.0
7,7,Franklin,franklin,NP0,,,0,a1e-fragment01,a1e,newspapers,Franklin,-1,a1e-fragment01,_,-1.0
8,10,leading,lead,VVG,mrw,met,0,a1e-fragment01,a1e,newspapers,leading,-1,a1e-fragment01,Leadership,-1.0
9,12,697m,697m,NN0,,,0,a1e-fragment01,a1e,newspapers,697m,-1,a1e-fragment01,_,-1.0


18527 18527
(18527, 7692)
Fitting 10 folds for each of 40 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:   

[LibLinear]newspapers
--(4988, 7691)


,word_id,word_x,lemma,word_type,function,seg_type,sentence_id,text_id_x,text_tag,genre,word_y,paragraph_id,text_id_y,frame,gwc
0,587,Letter,letter,NN1,,,30,a1f-fragment06,a1f,newspapers,Letter,-1,a1f-fragment06,_,-1.000000
1,588,Policy,policy,NN1,,,30,a1f-fragment06,a1f,newspapers,Policy,-1,a1f-fragment06,_,-1.000000
2,590,politics,politics,NN1,,,30,a1f-fragment06,a1f,newspapers,politics,-1,a1f-fragment06,_,-1.000000
3,593,Labour,labour,AJ0,,,30,a1f-fragment06,a1f,newspapers,Labour,-1,a1f-fragment06,_,-1.000000
4,594,Party,party,NN1,,,30,a1f-fragment06,a1f,newspapers,Party,-1,a1f-fragment06,Aggregate,-1.000000
5,595,Conference,conference,NN1,,,30,a1f-fragment06,a1f,newspapers,Conference,-1,a1f-fragment06,_,-1.000000
6,597,MR,mr,NP0,,,31,a1f-fragment06,a1f,newspapers,MR,-1,a1f-fragment06,_,-1.000000
7,598,DAVID,david,NP0,,,31,a1f-fragment06,a1f,newspapers,DAVID,-1,a1f-fragment06,_,-1.000000
8,599,SOSKIN,soskin,NP0-NN1,,,31,a1f-fragment06,a1f,newspapers,SOSKIN,-1,a1f-fragment06,_,-1.000000
9,600,Sir,sir,NN1,,,32,a1f-fragment06,a1f,newspapers,Sir,12,a1f-fragment06,_,-1.000000


4988 4988
(4988, 7692)
{'False': {'precision': 0.831729276622167, 'recall': 0.6734539969834088, 'f1-score': 0.7442700375052091, 'support': 3978}, 'True': {'precision': 0.26485568760611206, 'recall': 0.4633663366336634, 'f1-score': 0.33705437522506304, 'support': 1010}, 'micro avg': {'precision': 0.6309141940657578, 'recall': 0.6309141940657578, 'f1-score': 0.6309141940657578, 'support': 4988}, 'macro avg': {'precision': 0.5482924821141395, 'recall': 0.5684101668085361, 'f1-score': 0.5406622063651361, 'support': 4988}, 'weighted avg': {'precision': 0.7169453301694374, 'recall': 0.6309141940657578, 'f1-score': 0.6618145806281146, 'support': 4988}}
